<p><img src="imagenes/cabecera.png" width="900" align="center"></p>

# 1. Obtención de textos en español (libreta de curso)

## Curso Procesamiento de Lenguaje Natural 

### Maestría en Tecnologías de la información



#### Julio Waissman Vilanova (julio.waissman@unison.mx)


Uno de los problemas mas importantes a los que se enfrenta uno al desarrollar y aplicar técnicas para el Procesamiento de Lenguaje Natural (PLN) en español, es la falta de *corpus* disponibles que sirvan para la prueba y comparación de diferentes algoritmos. Por otra parte, dado que muchos de los algoritmo desarrollados para el PLN tienen interés al ser utilizados para analizar datos provenientes de redes sociales, es importante poder generar *corpus* provenientes de éstas a partir de la API ofrecida por las diferentes organizaciones y/o compañías.

Si bien esta no es una tarea propiamente de PLN, las técnicas modernas de PN requieren de datos de entrenamiento para poder realizar diferentes tareas. En esta libreta se muestran algunas técnicas sencillas para la obtención de diferentes *corpus* y su tratamiento básico en python, utilizando varias herramientas. Cabe resaltar que para aplicaciones más sofisticadas de PLN es necesario contar con *corpus* anotados (esto es, con información extra sobre temas, sentimientos y/o actitudes). Este tipo de *corpus* es dificil de obtener y generalmente se encuentran disponibles en venta, o son generados por las empresas para su utilización particular. Más tarde en en curso hablaremos sobre esto. 

## 1.1. Algunos *corpus* en español para descarga

Estos *corpus* (y su dirección web) pueden ser descargados directamente

1. [**Wikimedia dumps data en español**](https://dumps.wikimedia.org/eswiki/20180701/). Es la colección de datos 
   de *wikipedia*. La información viene en bruto, utilizando archivos en formato xml. Para más información sobre el
   formato de los archivos ver https://www.mediawiki.org/xml/export-0.8.xsd (en *Firefox* o *Chrome* de preferencia
   para poder navegar en la estructura). El conjunto completo de artículos se encuentran en
   https://dumps.wikimedia.org/eswiki/20180701/eswiki-20180701-pages-articles-multistream.xml.bz2 (3 Gb aprox.). 
   Utilizar de preferencia `curl` o `wget` dado el tamaño del archivo.
   
2. [**Wikipedia: servicio de exportación**](https://es.wikipedia.org/wiki/Especial:Exportar). En esta direccióon se
   encuentra una herramienta de exportación de Wikipedia para exportar entradas particulares. Es posible señalar
   categorías y la herramienta carga automáticamente las páginas asociadas a dicha categoría. Esta herramienta solo
   funciona con el nombre exacto de la categoría (o subcategoría). Un buen lugar para explorar las categorías de 
   *Wikipedia* es en su [página principal de portales](https://es.wikipedia.org/wiki/Portal:Portada). En particular, 
   la herramienta es importante para extraer información de categorás explicitas para entrenar a los métodos en 
   forma orientada.   

3. [**Spanish Billion Word Corpus (SBWC)**](https://crscardellino.github.io/SBWCE/). *Corpus* de documentos 
   recopilados y agregados por [Cristian Cardellino](https://crscardellino.github.io) de la Universidad Nacional de   
   Córdoba. Es una recopilación de documentos de diferentes fuentes disponibles gratuitamente en la web. El documento
   comprimido con el *corpus* completo se puede descargar [aqui](http://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/clean_corpus.tar.bz2) (2 Gb aprox.). Utilizar de preferencia `curl` o `wget` dado el tamaño del archivo.

4. [**Open subtitles**](http://opus.nlpl.eu/OpenSubtitles.php). Una colección importante de textos proveniente de 
   subtitulos de peliculas en diferentes idiomas. Muy util para entrenar traductores (incluye aliniamiento de frases 
   en diferentes idiomas), pero tambien incluye una fuente importante de frases en español. En particular, en 
   [esta página](http://opus.nlpl.eu/download.php?f=OpenSubtitles/en-es.txt.zip) se ecuentra un *corpus* con más de 500,000 frases en 
   español.
   
5. [**Conjunto de datos de TASS**](http://www.sepln.org/workshops/tass/2017/#datasets). Conjunto de datos de la 
   competencia organizada cada año en el *Taller de Análisis Semántico de la SEPLN (Sociedad Española de PLN)*. 
   A diferencia de los congresos en inglés, en los cuales los conjuntos de datos se abren a todo el público para
   servir de base de pruebas sin restricciones, la SEPLN solicita una petición expresa con un documento firmado 
   sobre el uso de los datos (y el compromiso de citar a los autores de la base de datos en cada artículo). 
   Cabe aclarar que los datos son gratuitos. Si pienasan utilizar los datos para un artículo, lo conveniente es
   solicitar los datos explicitamente a la SEPLN (en el link viene toda la información). Si los quieres utilizar 
   para probar algunas cosas, el conjunto de datos del 2015 se encuentran (por el momento) en una página personal 
   que subieron a *github* y desde ahí se pueden descargar. Se puede descargar aquí [los datos para entrenamiento 
   en general](https://raw.githubusercontent.com/imendibo/SEPLN-TASS15/master/DATA/general-tweets-train-tagged.xml), 
   y los [twits de prueba (sin etiquetar)](https://raw.githubusercontent.com/imendibo/SEPLN-TASS15/master/DATA/general-tweets-test.xml). 
   Igualmente, hay conjunto de datos para el análisis de sentimientos por aspectos, con [conjunto de entrenamiento](https://raw.githubusercontent.com/imendibo/SEPLN-TASS15/master/DATA/socialtv-tweets-train-tagged.xml) y 
   [conjunto de prueba](https://raw.githubusercontent.com/imendibo/SEPLN-TASS15/master/DATA/socialtv-tweets-test.xml).
   Por último, para análisis de sentimientos por aspecto y entidad ([conjunto de entrenamiento](https://raw.githubusercontent.com/imendibo/SEPLN-TASS15/master/DATA/stompol-tweets-train-tagged.xml), 
   [conjunto de prueba](https://raw.githubusercontent.com/imendibo/SEPLN-TASS15/master/DATA/stompol-tweets-train-tagged.xml)). 
   Estos últimos son muy valiosos debido a que han sido anotados. 


## 1.2. Procesamiento de los datos provenientes de *wikipedia*

Los datos provenientes de *wikipedia* vienen en formato `xml`, y el texto viene bajo un formato de lenguaje de marcado específico de wikipedia (muy similar a *markdown*). Es por eso que es muy importante extraer correctamente la información de los textos.

Supongamos que se utilizó la [herramienta de exportación de wikipedia](https://es.wikipedia.org/wiki/Especial:Exportar) en la categoría `Poetas en español` y el documento que se obtiene lo renombramos y guardamos en `datos/wikipedia-poetas-en-espanol.xml`. Una forma sencilla de limpiar el texto, es extraer los contenidos a traves de un generador en `python`. Vamos a crear un objeto iterador que nos permita ir generando los textos correspondientes conforme se requieren. Existen diferentes métodos para limpiar texto de *wikipedia*, algunos más extremos que otros, así que vamos a proponer 3, segun el grado de limpieza inicial que busquemos en los datos.

Este método no es el más elegante, pero nos permite procesar archivos xml de gran tamaño, sin tener que cargarlos completos en memoria.

In [ ]:
import xml.etree.ElementTree as et # Para manejar buffers con estructura xml
import codecs                       # Manejo de codificacion del texto
import re                           # Expresiones regulares
from bs4 import BeautifulSoup       # Tratamiento de etiquetas html

def generador_wiki_xml(archivo, metodo_limpieza='nada'):
    for event, elem in et.iterparse(archivo, events=('start', 'end', 'start-ns', 'end-ns')):
        if event == 'end':
            if elem.tag == '{http://www.mediawiki.org/xml/export-0.10/}page':
                elemento = {}
                for p in elem:
                    if p.tag == "{http://www.mediawiki.org/xml/export-0.10/}title":
                        elemento['titulo'] = p.text
                    elif p.tag == "{http://www.mediawiki.org/xml/export-0.10/}id":
                        elemento['id'] = int(p.text)
                    if p.tag == "{http://www.mediawiki.org/xml/export-0.10/}revision":
                        for x in p:
                            if x.tag == "{http://www.mediawiki.org/xml/export-0.10/}text": 
                                elemento['contenido'] = x.text
                if elemento['contenido'] is not None:
                    elemento['contenido'] = limpia_contenido(elemento['contenido'], metodo_limpieza)
                    yield elemento

    
def limpia_contenido(contenido, metodo_limpieza='nada'):
    if metodo_limpieza is 'nada':
        return contenido
    else:
        # Elimia algunas secciones no muy informativas de la wikipedia para PLN
        contenido = contenido[:contenido.find('== Véase también ==')]
        contenido = contenido[:contenido.find('== Referencias ==')]
        contenido = contenido[:contenido.find('== Notas y referencias ==')]
        contenido = contenido[:contenido.find('== Bibliografía ==')]
        contenido = contenido[:contenido.find('== Enlaces externos ==')]
        contenido = contenido[:contenido.find('== Obras ==')]
        contenido = contenido[:contenido.find('== Premios y distinciones ==')]
        contenido = contenido[:contenido.find('==Véase también==')]
        contenido = contenido[:contenido.find('==Referencias==')]
        contenido = contenido[:contenido.find('==Notas y referencias==')]
        contenido = contenido[:contenido.find('==Bibliografía==')]
        contenido = contenido[:contenido.find('==Enlaces externos==')]
        contenido = contenido[:contenido.find('==Obras==')]
        contenido = contenido[:contenido.find('==Premios y distinciones==')]

        contenido = contenido.lower()
        
        # Elimina las inserciones de xml, html y el marcado propio de wikipedia
        contenido = re.sub(r"{{[^(}})]*}}","", contenido)
        contenido = re.sub(r"\[\[[^\|\]]+\|", r"[[", contenido)
        contenido = re.sub(r"\[\[Archivo:[^(\]\])]*\]\]"," ", contenido)
        contenido = re.sub(r"\[\[File:[^(\]\])]*\]\]"," ", contenido)
        contenido = re.sub(r"\[\[Imagen:[^(\]\])]*\]\]"," ", contenido)
        contenido = re.sub(r"\[\[Image:[^(\]\])]*\]\]"," ", contenido)
        contenido = re.sub(r"\[\[Categoría:[^(\]\])]*\]\]"," ", contenido)
        contenido = BeautifulSoup(contenido, "lxml").get_text()

    if metodo_limpieza is 'maso':
        contenido = re.sub(r"<img([\w\W]+?)/?>", "", contenido)
        contenido = re.sub(r"\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*", r"#URL", contenido)
        contenido = re.sub(r"[\w\.-]+@[\w\.-]+", r"#EMAIL", contenido)
        contenido = re.sub(r"\[\[wikt[^|]*|", "", contenido)
        contenido = re.sub(r"<ref[^<]*<\/ref>","", contenido)
        contenido = re.sub(r"<.*>","", contenido)
        contenido = re.sub(r"&amp;", r"@", contenido)
        contenido = re.sub(r"&lt;", r"<", contenido)            
        contenido = re.sub(r"&gt;", r">", contenido)
        contenido = re.sub(r"&nbsp","", contenido)
        contenido = re.sub(r"\[\[","", contenido)
        contenido = re.sub(r"\]\]","", contenido)
        contenido = re.sub(r"'''",r'"', contenido)
        contenido = re.sub(r"''",r'"', contenido)
        contenido = re.sub(r"={3}.*={3}", "", contenido)
        contenido = re.sub(r"={2}.*={2}", "", contenido)
        contenido = re.sub(r"",r"", contenido)            
        contenido = re.sub(r"{{[^}}]*}}","", contenido)
        contenido = contenido.replace(u'\xa0', u' ')
    return contenido

y ahora vamos a probar con el archivo `datos/wikipedia-poetas.xml` para extraer los primeros textos. Es de resaltar que no compilamos las expresiones regulares, por lo que podría ser más lento que compilarlas cada una para su ejecución (en el modulo utils_docs.py viene compilado para poder utilizar las funciones en otras libretas). Y por supuesto, se puede generar un `Data Frame` de pandas y guardar el resultado en un archivo tipo `csv`.

In [ ]:
import pandas as pd
from random import randint
archivo_xml = 'datos/wikipedia-poetas.xml'
archivo_pkl = 'datos/wikipedia-poetas.pkl'

df = pd.DataFrame.from_dict(generador_wiki_xml(archivo_xml, 'maso'))
pd.to_pickle(df, archivo_pkl)

display(df.head(10))
i = randint(0, df.shape[0]-1)
print("El contenido formateado del documento {} es el siguiente:".format(i))
print(df.loc[i, 'contenido'])

Y hagamos lo mismo con un compendio (de Wikipedia) proveniente de la categoría *Políticos de Argentina*.

In [ ]:
archivo_xml = 'datos/wikipedia-politicos-argentina.xml'
archivo_pkl = 'datos/wikipedia-politicos-argentina.pkl'

df = pd.DataFrame.from_dict(generador_wiki_xml(archivo_xml, 'maso'))
pd.to_pickle(df, archivo_pkl)

display(df.head())
display(df.tail())
print("El contenido formateado del documento 394 es el siguiente:")
print(df.loc[394, 'contenido'])

## 1.3 Obtención de textos provenientes del *Proyecto Gutenberg*

El [Proyecto Gutenberg](https://www.gutenberg.org/wiki/ES_Portada) es una iniciativa para digitalizar y poner a disposición una gran cantidad de textos en diferentes idiomas, y por lo mismo es una fuente importante de datos para el procesamiento de lenguaje natural. Su principal debilidad es que solamente puede publicar libros sin derechos de autor, y por lo tanto son libros que normalmente suelen tener más de 100 años de su primera edición.

En esta sección vamos a mostrar como descargar libros desde *python*. Para seleccionar el libro, es necesario [navegar en el proyecto](https://www.gutenberg.org/browse/languages/es) para seleccionar las direcciones de los libros deseados.

In [ ]:
from urllib import request
import re

def recupera_pg(direccion, archivo):
    respuesta = request.urlopen(direccion)
    libro = respuesta.read().decode('utf8')  
    ind = libro.find("START OF THIS PROJECT GUTENBERG")
    libro = libro[ind:]
    ind = libro.find('\n')
    libro = libro[ind + 1:]
    ind = libro.find("*** END OF THIS PROJECT GUTENBERG")
    libro = libro[:ind]
    with open(archivo, 'w', encoding='utf8') as fp:
        fp.write(libro)
    return True

# La dirección de la obra "El Quijote" 
url = "http://www.gutenberg.org/cache/epub/2000/pg2000.txt"
archivo = "datos/quijote.txt"
recupera_pg(url, archivo)

# La dirección de "El Gaucho Martín Fierro"
url = "http://www.gutenberg.org/cache/epub/14765/pg14765.txt"
archivo = "datos/martin_fierro.txt"
recupera_pg(url, archivo)

## 1.4. Obtención de datos provenientes de *twitter*

Tweeter es posblemente la red social que mejor se presta para realizar PLN, y a partir de la cual se han creado varios *corpus* de prueba. En general, llos métodos para extracción de información en redes sociales funcionan más o menos igual, y por lo tanto ejemplificar el uso en *twitter* puede servir de idea de base para otras redes sociales. 

Para poder utilizar la interface de Twetter es necesario:

1. Contar con cuenta de *Twitter* y tener registrado el telefono (para la autentificación de dos tiempos).

2. Ir a la [página de desarrolladores](https://apps.twitter.com) (estando registrado en *Twetter*) y crear una nueva *App*. 

3. En la página generada de tu *App*, puedes recuperar en el apartado *Keys and Access Tokens* la información necesaria para poder recuperar información.

4. En esta misma página debes de crear tu *Access Token* para que tu *App* pueda acceder a la información (Una clave por desarrollador y una clave por aplicación).

La información la puedes recuperar fuera de linea, o en linea. Vamos a presentar las dos opciones.

Para recuperar la información fuera de linea se utiliza la API más sencilla. En el ejemplo siguiente, deberías tener el texto y el autor de los últimos 100 *tweets* que recibiste en tu cuenta:


In [ ]:
import tweepy

consumer_key = 'PN2ri0UvsLYawitO8Hako1PvM'
consumer_secret = 'fud3LXwVza6PQMAIRmA9Y9mJ6Gc2Y1uNkV3f8G2okPZHXrCdTK'
access_token = '43091680-zO6sw8j0VxjN2df1L3fUqMCFoRr78iNfq4eNtirqh'
access_token_secret = 'dt3eeJgi6fzFHzalwrH7oq2B624THUTEjhiqzrqpL9BzH'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

boca_tweets = api.user_timeline('@BocaJrsOficial', count=10)
river_tweets = api.user_timeline('@CARPoficial', count=10)

for tweet in boca_tweets + river_tweets:
    print(tweet.author.name + ': ')
    print(tweet.text)
    print(20*'=')

Tambien se pueden buscar los trending topics (globales, por país o por zona). Esto es útil si queremos hacer una base de datos con la evolución de los temas en el tiempo.

In [ ]:
consumer_key = 'PN2ri0UvsLYawitO8Hako1PvM'
consumer_secret = 'fud3LXwVza6PQMAIRmA9Y9mJ6Gc2Y1uNkV3f8G2okPZHXrCdTK'
access_token = '43091680-zO6sw8j0VxjN2df1L3fUqMCFoRr78iNfq4eNtirqh'
access_token_secret = 'dt3eeJgi6fzFHzalwrH7oq2B624THUTEjhiqzrqpL9BzH'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

mx_trends = api.trends_place(23424900)[0]
ar_trends = api.trends_place(23424747)[0]

print("\n\n********* Trends en México ***********")
for trend in mx_trends['trends'][:10]:
    print(10*'=')
    print('Titulo: ' + trend['name'])
    print('URL: ' + trend['url'])

print("\n\n********* Trends en Argentina ***********")
for trend in ar_trends['trends'][:10]:
    print(10*'=')
    print('Titulo: ' + trend['name'])
    print('URL: ' + trend['url'])

Si queremos recuperar (y posiblemente procesar) *tweets* en linea  tenemos que hacer algo un poco más complejo. Este script hay que dejarlo ejecutar por un tiempo y luego hay que detener la ejecución de la celda (botón de paro en la barra de herramientas).

In [ ]:
import json
import tweepy

consumer_key = 'PN2ri0UvsLYawitO8Hako1PvM'
consumer_secret = 'fud3LXwVza6PQMAIRmA9Y9mJ6Gc2Y1uNkV3f8G2okPZHXrCdTK'
access_token = '43091680-zO6sw8j0VxjN2df1L3fUqMCFoRr78iNfq4eNtirqh'
access_token_secret = 'dt3eeJgi6fzFHzalwrH7oq2B624THUTEjhiqzrqpL9BzH'

archivo = 'datos/tweets_ejemplo.csv'

class Listener(tweepy.streaming.StreamListener):

    def on_data(self, data):
        try:
            tweet = json.loads(data)
        except Exception as e:
            print(e) 
            return True

        texto = tweet['text'].replace('\n', ' ').replace('\t', ' ')
        usuario = '@' + tweet.get('user').get('screen_name')
        fecha = tweet.get('created_at')
        tweet = u'{}\t{}\t{}\n'.format(texto, usuario, fecha)
        
        self.archivo.write(tweet)
        return True

    def on_error(self, status):
        print(status)

with  open(archivo, 'a', encoding='utf-8') as fp:
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    
    listner = Listener()
    listner.archivo = fp
    twitterStream = tweepy.Stream(auth, listner)
    
    twitterStream.filter(track=['AMLO', 'MORENA', 'Lopez Obrador'])    

Cada vez que volvamos a ejecutar el script anetrios aumentará el *corpus* que estamos creando.

Para el filtro se pueden utilizar diferentes criterios:

- track: Una lista con frases o palabras que deben encontrarse en el texto del tweet
- folow: Una lista con IDs de usuarios a esperar sus *tweets*
- locations: Una lista con cuatro valores delimitando la región donde se buscarán tweets

El archivo que generamos, lo podemos leer ahora como un `DataTable` de *pandas*. 

In [ ]:
import pandas as pd

archivo = 'datos/tweets_ejemplo.csv'
dt = pd.read_csv(archivo, sep='\t', header=None, names=['Texto', 'Usuario', 'Creado el'])
dt.sample(20)

## 1.5. Procesamiento de los datos TASS 2015

Con el fin de completar el proceso de tratamiento de información, vamos a generar un `DataFrame` en *pandas* con el fin de utilizarlo posteriormente en las tareas de PLN subsecuentes del conjunto de datos más sencillo (únicamente polaridad general de cada *tweet*. Los archivos vienen en formato *xml* por lo que su procesamiento es relativamente simple. 

In [ ]:
import xml.etree.ElementTree as et
import pandas as pd

archivo_train = "datos/tass2015/general-tweets-train-tagged.xml"
pickle_train = 'datos/tass2015/general-tweets-train-dt.pkl'
archivo_test = "datos/tass2015/general-tweets-test.xml"
pickle_test = 'datos/tass2015/general-tweets-test-dt.pkl'

arbol = et.parse(archivo_train)
raiz = arbol.getroot()
data_dic = []
for tweet in raiz.iter('tweet'):
    contenido = tweet.find('content').text
    if contenido is not None:
        data_dic.append({
            'texto': contenido,
            'polaridad': tweet.find('sentiments')[0].find('value').text,
            'id': tweet.find('tweetid').text,
            'usuario': '@' + tweet.find('user').text,
            'fecha': tweet.find('date').text,
            'tópicos': '[' + ', '.join(['"' + t.text + '"' for t in tweet.find('topics')]) + ']'
        })
df_train = pd.DataFrame.from_dict(data_dic)
display(df_train.head(10))
pd.to_pickle(df_train, pickle_train)

arbol = et.parse(archivo_test)
raiz = arbol.getroot()
data_dic = []
for tweet in raiz.iter('tweet'):
    contenido = tweet.find('content').text
    if contenido is not None:
        data_dic.append({
            'texto': contenido,
            'id': tweet.find('tweetid').text,
            'usuario': '@' + tweet.find('user').text,
            'fecha': tweet.find('date').text,
        })
df_test = pd.DataFrame.from_dict(data_dic)
display(df_test.head(10))
pd.to_pickle(df_train, pickle_test)